In [1]:
from pydantic import BaseModel, Field
from langchain_tavily import TavilySearch
from langchain.chat_models import init_chat_model

from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

c:\RAG\learning\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class MovieReview(BaseModel):
    """ Class is used by LLM to get Movie Review """

    title: str = Field(max_length=500, description="Define the Movie Name")
    hero: str = Field(max_length=500, description="Provide the name of Lead Male Role")
    heroin: str = Field(max_length=500, description="Provide the name of Lead Female Role")
    year: int = Field( description= "Provide Year of Movie Release Date")
    revenue: float = Field( default="Define total world wide revenue in dollors")
    makingCost: float = Field(  default="total Making cost of Movie")
    hitOrFlot: str = Field(description=" Movie is Hit or Flop")
    ratting: float = Field(ge=0.0, le=10.0, description="get IMDB ratting")

In [3]:
class MovieList(BaseModel):
    """ Provide Movies in List """

    movies: list[MovieReview] = Field(description="Field Provides list of Movie Reviews")

In [4]:
wiki_tool =  WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(
                                                        top_k_results=3,
                                                        doc_content_chars_max=2000
                                                        )
                               )

In [5]:
tavily_tool = TavilySearch(
    max_results = 5,
    topic = "general"
)

In [6]:
llm = init_chat_model(
    model="gpt-4o-mini",
    model_provider="openai"
)
llm_travily = llm.bind_tools([ wiki_tool])

In [7]:
model = llm_travily.with_structured_output(MovieList)

In [14]:
result = model.invoke("tell me about worst 5 movies of 2024")

In [15]:
for item in result.movies:
    print(item)

title='The Space Chronicles' hero='Sam Worthington' heroin='Lily Collins' year=2024 revenue=2000000.0 makingCost=150000000.0 hitOrFlot='Flop' ratting=4.1
title='Ghost Train' hero='John Boyega' heroin='Emma Watson' year=2024 revenue=1500000.0 makingCost=80000000.0 hitOrFlot='Flop' ratting=3.8
title='Love in the Shadows' hero='Tom Holland' heroin='Zendaya' year=2024 revenue=3000000.0 makingCost=60000000.0 hitOrFlot='Flop' ratting=4.0
title='Battle of the Giants' hero='Vin Diesel' heroin='Charlize Theron' year=2024 revenue=4000000.0 makingCost=100000000.0 hitOrFlot='Flop' ratting=3.9
title='Digital Dreams' hero='Chris Pratt' heroin='Margot Robbie' year=2024 revenue=2500000.0 makingCost=90000000.0 hitOrFlot='Flop' ratting=4.2
